In [1]:
import pandas as pd
import datetime as dt

In [2]:
dataset=pd.read_csv("Chiller con week days.csv")

In [3]:
dataset

,DAYS,DATE,TOTAL A,TOTAL B,TOTAL C
0,Monday,2023-10-09,8.96,7.84,5.80
1,Tuesday,2023-10-10,10.82,9.87,5.37
2,Wednesday,2023-10-11,9.46,9.46,6.07
3,Thursday,2023-10-12,9.26,11.12,4.38
4,Friday,2023-10-13,8.27,11.00,3.94
...,...,...,...,...,...
84,Wednesday,2024-02-07,7.81,8.62,2.86
85,Thursday,2024-02-08,5.37,4.56,2.97
86,Friday,2024-02-09,6.86,7.89,2.67
87,Monday,2024-02-12,3.02,1.49,1.56


In [4]:
dataset.columns

Index(['DAYS', 'DATE', 'TOTAL A', 'TOTAL B', 'TOTAL C'], dtype='object')

In [5]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DAYS     89 non-null     object 
 1   DATE     89 non-null     object 
 2   TOTAL A  89 non-null     float64
 3   TOTAL B  89 non-null     float64
 4   TOTAL C  89 non-null     float64
dtypes: float64(3), object(2)
memory usage: 3.6+ KB


In [6]:
dataset['DATE']=pd.to_datetime(dataset['DATE'])

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   DAYS     89 non-null     object        
 1   DATE     89 non-null     datetime64[ns]
 2   TOTAL A  89 non-null     float64       
 3   TOTAL B  89 non-null     float64       
 4   TOTAL C  89 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 3.6+ KB


In [9]:
startdate="2023-10-09"
enddate="2024-02-13"
df=dataset[dataset['DATE'].between(startdate,enddate)][['DATE', 'TOTAL C']]

In [10]:
df

,DATE,TOTAL C
0,2023-10-09,5.80
1,2023-10-10,5.37
2,2023-10-11,6.07
3,2023-10-12,4.38
4,2023-10-13,3.94
...,...,...
84,2024-02-07,2.86
85,2024-02-08,2.97
86,2024-02-09,2.67
87,2024-02-12,1.56


In [11]:
column='TOTAL C'

In [12]:
from sklearn.preprocessing import MinMaxScaler
Ms=MinMaxScaler()
data1=Ms.fit_transform(df[[column]])
print("Len:",data1.shape)

Len: (89, 1)


In [13]:
len(data1)

89

In [14]:
training_size=round(len(data1)*0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

71
X_train length: (71, 1)
X_test length: (18, 1)
y_train length: (71, 1)
y_test length: (18, 1)


In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
#pip install statsmodels

In [17]:
from sklearn.metrics import mean_squared_error
trends=['n','t','c','ct']
orders=[(0,0,1),(0,0,2),(0,0,3)]
from statsmodels.tsa.arima.model import ARIMA
for td in trends:
        
        #print(td)
        #trendslist.append(td)
    model = ARIMA(X_train, order=(0,0,10),trend=td,)
    model_fit = model.fit()
        # make prediction
    y_pred= model_fit.predict(len(X_train), len(data1)-1)
        #print(y_pred)
    from sklearn.metrics import r2_score
    mse=mean_squared_error(y_test,y_pred,squared=False)
    from stockFunctions import rmsemape
    print("Trend={}".format(td))
    rmsemape(y_test,y_pred)
    print("************")

C:\Users\Hi\anaconda3\envs\aiads\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Trend=n
RMSE-Testset: 0.3299245909886484
maPe-Testset: 43732424677296.33
************


C:\Users\Hi\anaconda3\envs\aiads\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Trend=t
RMSE-Testset: 0.5710104963991345
maPe-Testset: 253056296965851.12
************


C:\Users\Hi\anaconda3\envs\aiads\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Trend=c
RMSE-Testset: 0.25223757491974713
maPe-Testset: 155702743039660.0
************
Trend=ct
RMSE-Testset: 0.17240456217013278
maPe-Testset: 115536387315739.16
************


In [18]:
len(y_test)

18

In [19]:
#i=1
td="ct"
model = ARIMA(X_train, order=(0,0,30),trend=td)
model_fit = model.fit()
# make prediction
y_pred= model_fit.predict(len(X_train), len(data1)-1)
#print(y_pred)
from sklearn.metrics import r2_score
mse=mean_squared_error(y_test,y_pred,squared=False)
from stockFunctions import rmsemape
print("Trend={}".format(td))
rmsemape(y_test,y_pred)
print("************")

Trend=ct
RMSE-Testset: 0.17200511864427231
maPe-Testset: 81950707800265.94
************


In [20]:
#pip install Pillow


In [21]:
#from stockFunctions import graph
#graph(y_test,y_pred,"Actual","predicted","Quality data","Time","Char A")

In [22]:
len(data1)

89

In [23]:
y_test

array([[0.66886326],
       [0.45469522],
       [0.53871499],
       [0.51565074],
       [0.4645799 ],
       [0.44810544],
       [0.32125206],
       [0.3261944 ],
       [0.        ],
       [0.46622735],
       [0.57990115],
       [0.31960461],
       [0.1878089 ],
       [0.47116969],
       [0.4892916 ],
       [0.4398682 ],
       [0.25700165],
       [0.66886326]])

In [24]:
from stockFunctions import conversionSingle
aTestNormTable=conversionSingle(y_test,[column])

In [25]:
aTestNormTable

,TOTAL C
0,0.668863
1,0.454695
2,0.538715
3,0.515651
4,0.46458
5,0.448105
6,0.321252
7,0.326194
8,0.0
9,0.466227


In [26]:
actual_stock_price_test_ori=Ms.inverse_transform(aTestNormTable)

In [27]:
actual_stock_price_test_ori

array([[4.06],
       [2.76],
       [3.27],
       [3.13],
       [2.82],
       [2.72],
       [1.95],
       [1.98],
       [0.  ],
       [2.83],
       [3.52],
       [1.94],
       [1.14],
       [2.86],
       [2.97],
       [2.67],
       [1.56],
       [4.06]])

In [28]:
actual_stock_price_test_oriA=conversionSingle(actual_stock_price_test_ori,[column])

In [29]:
actual_stock_price_test_oriA

,TOTAL C
0,4.06
1,2.76
2,3.27
3,3.13
4,2.82
5,2.72
6,1.95
7,1.98
8,0.0
9,2.83


In [30]:
from stockFunctions import conversionSingle
aTestNormTable=conversionSingle(y_test,[column])
actual_stock_price_test_ori=Ms.inverse_transform(aTestNormTable)
actual_stock_price_test_oriA=conversionSingle(actual_stock_price_test_ori,[column])

In [31]:
from stockFunctions import conversionSingle
pTestNormTable=conversionSingle(y_pred,[column])
predicted_stock_price_test_ori=Ms.inverse_transform(pTestNormTable)
predicted_stock_price_test_oriP=conversionSingle(predicted_stock_price_test_ori,[column])

In [32]:
#from stockFunctions import graph
#graph(actual_stock_price_test_oriA,predicted_stock_price_test_oriP,"Actual","Predicted","Quality data","Time","Char A")

In [33]:
from stockFunctions import rmsemape
rmsemape(actual_stock_price_test_oriA,predicted_stock_price_test_oriP)

RMSE-Testset: 1.044071070170733
maPe-Testset: 497440796347612.6


In [34]:
forecast=model_fit.predict(len(data1), len(data1)+10)

In [35]:
forecast

array([0.40293711, 0.3736193 , 0.44617196, 0.38839547, 0.31491545,
       0.48757781, 0.29588321, 0.39326301, 0.39330239, 0.34978201,
       0.38119096])

In [36]:
from stockFunctions import conversionSingle
fTestNormTable=conversionSingle(forecast,["TOTAL C"])
forecast_stock_price_test_ori=Ms.inverse_transform(fTestNormTable)
forecast_stock_price_test_oriF=conversionSingle(forecast_stock_price_test_ori,["TOTAL C fore cast"])

In [37]:
forecast_stock_price_test_oriF

,TOTAL C fore cast
0,2.445828
1,2.267869
2,2.708264
3,2.35756
4,1.911537
5,2.959597
6,1.796011
7,2.387106
8,2.387345
9,2.123177
